## Mount the google colab

In [ ]:
# from google.colab import drive
# drive.mount('/gdrive')
# %cd /gdrive/MyDrive/videos_all

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

## Install required packages

In [ ]:
%pip install keras-video-generators
!pip install keras-tuner

## Import required pacakges and their classes

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, TimeDistributed, LSTM, Dropout, GRU, Flatten, Input, ConvLSTM2D, BatchNormalization, MaxPool3D, GaussianNoise
from tensorflow.keras.models import Model
from keras_video import VideoFrameGenerator
from tensorflow.keras.applications import ResNet50, ResNet50V2
from kerastuner import RandomSearch, BayesianOptimization
import tensorflow as tf
from tensorflow.keras.utils import plot_model 

### Path for the videos 

In [ ]:
pattern = '/content/drive/MyDrive/videos_all/{classname}/*.mp4'

## Generate video frames 

In [ ]:
tr_data = VideoFrameGenerator(
          classes = ['confused', 'not_confused'],
          glob_pattern = pattern,
          nb_frames = 20,
          split_val = 0.25,
          shuffle = False,
          batch_size = 3,
          target_shape = (224, 224),
          nb_channel = 3,
          use_frame_cache = False
)

va_data = tr_data.get_validation_generator()

In [ ]:
# tr_data = VideoFrameGenerator(
#           classes = ['confused', 'not_confused', 'uncertain'],
#           glob_pattern = pattern,
#           nb_frames = 20,
#           split_val = 0.25,
#           shuffle = False,
#           batch_size = 3,
#           target_shape = (224, 224),
#           nb_channel = 3,
#           use_frame_cache = False
# )

# va_data = tr_data.get_validation_generator()

## Define Model Structures for 2 classes

In [ ]:
# change the nb fram input from 10 to 20
# change the output from 3 to 2
# change the decay steps num
tr_decay =392
def convlstm(hp):
    """
        Network structure extra with ConvLSTM layer.

    """
    activation = hp.Choice('activation', ['relu', 'tanh'])
    dropout_rate = hp.Float('dropout_rate', min_value=0.1, max_value=0.9, step=0.2)

    model = Sequential()
    model.add(ConvLSTM2D(filters = 48, kernel_size = (3, 3), return_sequences = True, data_format = "channels_last", input_shape = (20,224,224,3)))
    model.add(GaussianNoise(0.02))
    model.add(ConvLSTM2D(filters = 32, kernel_size = (3, 3), return_sequences = True, data_format = "channels_last"))
    model.add(GaussianNoise(0.02))
    model.add(ConvLSTM2D(filters = 16, kernel_size = (3, 3), return_sequences = False, data_format = "channels_last"))
    model.add(MaxPool2D(pool_size=(2,2), data_format='channels_last'))
    model.add(Dropout(dropout_rate))
    model.add(Flatten())
    model.add(Dense(48, activation = activation))
    model.add(Dropout(0.4))
    model.add(Dense(24, activation = activation))
    model.add(Dense(2, activation = 'softmax'))

    lr = hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])
    decay_rate = hp.Choice('decay_rate', [0.90, 0.60, 0.30, 0.10])
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=lr, decay_rate=decay_rate, decay_steps = tr_decay/tr_data.batch_size, staircase=True)

    # optimizer = hp.Fixed('optimizer', ['sgd'])
    model.compile(optimizer=SGD(lr_schedule), loss='categorical_crossentropy', metrics=['accuracy'])
    return model


def resnetlstmgru(hp):
    """
        Network structure extra with ResNet50v2 + LSTM + GRU. 

    """

    activation = hp.Choice('activation', ['relu', 'tanh'])
    dropout_rate = hp.Float('dropout_rate', min_value=0.1, max_value=0.9, step=0.2)

    model = Sequential()
    model.add(TimeDistributed(ResNet50V2(include_top=False, weights='imagenet', pooling='max'), input_shape=(20,224,224,3)))
    # model.add(TimeDistributed(Conv2D(filters = 64, kernel_size = (3,3), padding = 'same', activation = 'relu')))
    # model.add(TimeDistributed(MaxPool2D(pool_size = (2,2))))
    model.add(TimeDistributed(Flatten()))
    model.add(LSTM(10, return_sequences = True))
    model.add(GRU(10))
    model.add(Dense(48, activation = activation))
    model.add(Dropout(dropout_rate))
    model.add(Dense(24, activation = activation))
    model.add(Dense(2, activation = 'softmax'))

    lr = hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])
    decay_rate = hp.Choice('decay_rate', [0.90, 0.60, 0.30, 0.10])
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=lr, decay_rate=decay_rate, decay_steps = tr_decay/tr_data.batch_size, staircase=True)

    # optimizer = hp.Fixed('optimizer', ['sgd'])
    model.compile(optimizer=SGD(lr_schedule), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def cnnlstmgru(hp):
    """
        Network structure with CNN + LSTM + GRU.

    """

    activation = hp.Choice('activation', ['relu', 'tanh'])
    dropout_rate = hp.Float('dropout_rate', min_value=0.1, max_value=0.9, step=0.2)

    model = Sequential()
    model.add(TimeDistributed(Conv2D(filters = 64, kernel_size = (3,3), padding = 'same', activation = activation), input_shape = (20, 224, 224, 3)))
    model.add(TimeDistributed(MaxPool2D(pool_size = (2,2))))
    model.add(TimeDistributed(Flatten()))
    model.add(LSTM(10, return_sequences = True))
    model.add(GRU(10))
    model.add(Dense(48, activation = activation))
    model.add(Dropout(dropout_rate))
    model.add(Dense(24, activation = activation))
    model.add(Dense(2, activation = 'softmax'))

    lr = hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])
    decay_rate = hp.Choice('decay_rate', [0.90, 0.60, 0.30, 0.10])
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=lr, decay_rate=decay_rate, decay_steps = tr_decay/tr_data.batch_size, staircase=True)

    # optimizer = hp.Fixed('optimizer', ['sgd'])
    model.compile(optimizer=SGD(lr_schedule), loss='categorical_crossentropy', metrics=['accuracy'])
    return model


def cnnlstm(hp):
    """
        Network structure with CNN + LSTM + GRU.

    """

    activation = hp.Choice('activation', ['relu', 'tanh'])
    dropout_rate = hp.Float('dropout_rate', min_value=0.1, max_value=0.9, step=0.2)

    model = Sequential()
    model.add(TimeDistributed(Conv2D(filters = 64, kernel_size = (3,3), padding = 'same', activation = activation), input_shape = (20, 224, 224, 3)))
    model.add(TimeDistributed(MaxPool2D(pool_size = (2,2))))
    model.add(TimeDistributed(Flatten()))
    model.add(LSTM(10, return_sequences = False))
    model.add(Dense(48, activation = activation))
    model.add(Dropout(dropout_rate))
    model.add(Dense(24, activation = activation))
    model.add(Dense(2, activation = 'softmax'))

    lr = hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])
    decay_rate = hp.Choice('decay_rate', [0.90, 0.60, 0.30, 0.10])
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=lr, decay_rate=decay_rate, decay_steps = tr_decay/tr_data.batch_size, staircase=True)

    # optimizer = hp.Fixed('optimizer', ['sgd'])
    model.compile(optimizer=SGD(lr_schedule), loss='categorical_crossentropy', metrics=['accuracy'])
    return model


def cnngru(hp):
    """
        Network structure with CNN + LSTM + GRU.

    """

    activation = hp.Choice('activation', ['relu', 'tanh'])
    dropout_rate = hp.Float('dropout_rate', min_value=0.1, max_value=0.9, step=0.2)

    model = Sequential()
    model.add(TimeDistributed(Conv2D(filters = 64, kernel_size = (3,3), padding = 'same', activation = activation), input_shape = (20, 224, 224, 3)))
    model.add(TimeDistributed(MaxPool2D(pool_size = (2,2))))
    model.add(TimeDistributed(Flatten()))
    model.add(GRU(10, return_sequences=False))
    model.add(Dense(48, activation = activation))
    model.add(Dropout(dropout_rate))
    model.add(Dense(24, activation = activation))
    model.add(Dense(2, activation = 'softmax'))

    lr = hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])
    decay_rate = hp.Choice('decay_rate', [0.90, 0.60, 0.30, 0.10])
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=lr, decay_rate=decay_rate, decay_steps = tr_decay/tr_data.batch_size, staircase=True)

    # optimizer = hp.Fixed('optimizer', ['sgd'])
    model.compile(optimizer=SGD(lr_schedule), loss='categorical_crossentropy', metrics=['accuracy'])
    return model



## Define Model structure for 3 classes

In [ ]:

def convlstm(hp):
    """
        Network structure extra with ConvLSTM layer.

    """
    activation = hp.Choice('activation', ['relu', 'tanh'])
    dropout_rate = hp.Float('dropout_rate', min_value=0.1, max_value=0.9, step=0.2)

    model = Sequential()
    model.add(ConvLSTM2D(filters = 48, kernel_size = (3, 3), return_sequences = True, data_format = "channels_last", input_shape = (10,224,224,3)))
    model.add(GaussianNoise(0.02))
    model.add(ConvLSTM2D(filters = 32, kernel_size = (3, 3), return_sequences = True, data_format = "channels_last"))
    model.add(GaussianNoise(0.02))
    model.add(ConvLSTM2D(filters = 16, kernel_size = (3, 3), return_sequences = False, data_format = "channels_last"))
    model.add(MaxPool2D(pool_size=(2,2), data_format='channels_last'))
    model.add(Dropout(dropout_rate))
    model.add(Flatten())
    model.add(Dense(48, activation = activation))
    model.add(Dropout(0.4))
    model.add(Dense(24, activation = activation))
    model.add(Dense(3, activation = 'softmax'))

    lr = hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])
    decay_rate = hp.Choice('decay_rate', [0.90, 0.60, 0.30, 0.10])
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=lr, decay_rate=decay_rate, decay_steps = 605/tr_data.batch_size, staircase=True)

    # optimizer = hp.Fixed('optimizer', ['sgd'])
    model.compile(optimizer=SGD(lr_schedule), loss='categorical_crossentropy', metrics=['accuracy'])
    model.summary()
    return model


def resnetlstmgru(hp):
    """
        Network structure extra with ResNet50v2 + LSTM + GRU. 

    """

    activation = hp.Choice('activation', ['relu', 'tanh'])
    dropout_rate = hp.Float('dropout_rate', min_value=0.1, max_value=0.9, step=0.2)

    model = Sequential()
    model.add(TimeDistributed(ResNet50V2(include_top=False, weights='imagenet', pooling='max'), input_shape=(10,224,224,3)))
    # model.add(TimeDistributed(Conv2D(filters = 64, kernel_size = (3,3), padding = 'same', activation = 'relu')))
    # model.add(TimeDistributed(MaxPool2D(pool_size = (2,2))))
    model.add(TimeDistributed(Flatten()))
    model.add(LSTM(10, return_sequences = True))
    model.add(GRU(10))
    model.add(Dense(48, activation = activation))
    model.add(Dropout(dropout_rate))
    model.add(Dense(24, activation = activation))
    model.add(Dense(3, activation = 'softmax'))

    lr = hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])
    decay_rate = hp.Choice('decay_rate', [0.90, 0.60, 0.30, 0.10])
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=lr, decay_rate=decay_rate, decay_steps = 605/tr_data.batch_size, staircase=True)

    # optimizer = hp.Fixed('optimizer', ['sgd'])
    model.compile(optimizer=SGD(lr_schedule), loss='categorical_crossentropy', metrics=['accuracy'])
    model.summary()
    return model

def cnnlstmgru(hp):
    """
        Network structure with CNN + LSTM + GRU.

    """

    activation = hp.Choice('activation', ['relu', 'tanh'])
    dropout_rate = hp.Float('dropout_rate', min_value=0.1, max_value=0.9, step=0.2)

    model = Sequential()
    model.add(TimeDistributed(Conv2D(filters = 64, kernel_size = (3,3), padding = 'same', activation = activation), input_shape = (10, 224, 224, 3)))
    model.add(TimeDistributed(MaxPool2D(pool_size = (2,2))))
    model.add(TimeDistributed(Flatten()))
    model.add(LSTM(10, return_sequences = True))
    model.add(GRU(10))
    model.add(Dense(48, activation = activation))
    model.add(Dropout(dropout_rate))
    model.add(Dense(24, activation = activation))
    model.add(Dense(3, activation = 'softmax'))

    lr = hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])
    decay_rate = hp.Choice('decay_rate', [0.90, 0.60, 0.30, 0.10])
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=lr, decay_rate=decay_rate, decay_steps = 605/tr_data.batch_size, staircase=True)

    # optimizer = hp.Fixed('optimizer', ['sgd'])
    model.compile(optimizer=SGD(lr_schedule), loss='categorical_crossentropy', metrics=['accuracy'])
    model.summary()
    return model


def cnnlstm(hp):
    """
        Network structure with CNN + LSTM + GRU.

    """

    activation = hp.Choice('activation', ['relu', 'tanh'])
    dropout_rate = hp.Float('dropout_rate', min_value=0.1, max_value=0.9, step=0.2)

    model = Sequential()
    model.add(TimeDistributed(Conv2D(filters = 64, kernel_size = (3,3), padding = 'same', activation = activation), input_shape = (10, 224, 224, 3)))
    model.add(TimeDistributed(MaxPool2D(pool_size = (2,2))))
    model.add(TimeDistributed(Flatten()))
    model.add(LSTM(10, return_sequences = False))
    model.add(Dense(48, activation = activation))
    model.add(Dropout(dropout_rate))
    model.add(Dense(24, activation = activation))
    model.add(Dense(3, activation = 'softmax'))

    lr = hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])
    decay_rate = hp.Choice('decay_rate', [0.90, 0.60, 0.30, 0.10])
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=lr, decay_rate=decay_rate, decay_steps = 605/tr_data.batch_size, staircase=True)

    # optimizer = hp.Fixed('optimizer', ['sgd'])
    model.compile(optimizer=SGD(lr_schedule), loss='categorical_crossentropy', metrics=['accuracy'])
    model.summary()
    return model


def cnngru(hp):
    """
        Network structure with CNN + LSTM + GRU.

    """

    activation = hp.Choice('activation', ['relu', 'tanh'])
    dropout_rate = hp.Float('dropout_rate', min_value=0.1, max_value=0.9, step=0.2)

    model = Sequential()
    model.add(TimeDistributed(Conv2D(filters = 64, kernel_size = (3,3), padding = 'same', activation = activation), input_shape = (10, 224, 224, 3)))
    model.add(TimeDistributed(MaxPool2D(pool_size = (2,2))))
    model.add(TimeDistributed(Flatten()))
    model.add(GRU(10, return_sequences=False))
    model.add(Dense(48, activation = activation))
    model.add(Dropout(dropout_rate))
    model.add(Dense(24, activation = activation))
    model.add(Dense(3, activation = 'softmax'))

    lr = hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])
    decay_rate = hp.Choice('decay_rate', [0.90, 0.60, 0.30, 0.10])
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=lr, decay_rate=decay_rate, decay_steps = 605/tr_data.batch_size, staircase=True)

    # optimizer = hp.Fixed('optimizer', ['sgd'])
    model.compile(optimizer=SGD(lr_schedule), loss='categorical_crossentropy', metrics=['accuracy'])
    model.summary()
    return model



## Find the best hyperparameters using RandomSearch

> `RandomSearch(define_model)` : Replace `define_model` parameter with name of the model function you want to train.



In [ ]:
# model1: cnngru
import datetime
MAX_TRAILS = 66
epochs = 10
seed = 40

# change the first parameter according to the network you want to train. 
tuner = RandomSearch(cnngru,
                     objective='val_accuracy',
                     max_trials=MAX_TRAILS,
                     seed=seed,
                     overwrite=True
                     )

print("Search space summary:\n {}".format(tuner.search_space_summary()))
print("Searching starting >>> started")

tr_decay = 392
va_decay = 130
dir = '/content/drive/MyDrive/cnngru/logs/tuner/' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tuner.search(tr_data, epochs=epochs, steps_per_epoch=tr_decay//tr_data.batch_size,
             validation_data=va_data, verbose=1, validation_steps=va_decay//va_data.batch_size, 
             callbacks=[tf.keras.callbacks.TensorBoard(dir)])

print("Results summary:\n {}".format(tuner.results_summary()))
print("Top 10 best models:\n {}".format(tuner.get_best_models(10)))

best_hyperparameters = tuner.get_best_hyperparameters(num_trials = 1)[0]
best_model = tuner.hypermodel.build(best_hyperparameters)

best_model.save('/content/drive/MyDrive/cnngru/Model/')


In [ ]:
# model 2: cnnlstm
import datetime
MAX_TRAILS = 66
epochs = 10
seed = 40

# change the first parameter according to the network you want to train. 
tuner = RandomSearch(cnnlstm,
                     objective='val_accuracy',
                     max_trials=MAX_TRAILS,
                     seed=seed,
                     overwrite=True
                     )

print("Search space summary:\n {}".format(tuner.search_space_summary()))
print("Searching starting >>> started")

tr_decay = 392
va_decay = 130
dir = '/content/drive/MyDrive/cnnlstm/logs/tuner/' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tuner.search(tr_data, epochs=epochs, steps_per_epoch=tr_decay//tr_data.batch_size,
             validation_data=va_data, verbose=1, validation_steps=va_decay//va_data.batch_size, 
             callbacks=[tf.keras.callbacks.TensorBoard(dir)])

print("Results summary:\n {}".format(tuner.results_summary()))
print("Top 10 best models:\n {}".format(tuner.get_best_models(10)))

best_hyperparameters = tuner.get_best_hyperparameters(num_trials = 1)[0]
best_model = tuner.hypermodel.build(best_hyperparameters)

best_model.save('/content/drive/MyDrive/cnnlstm/Model/')


In [ ]:
# model 3: resnetlstmgru
import datetime
MAX_TRAILS = 66
epochs = 10
seed = 40

# change the first parameter according to the network you want to train. 
tuner = RandomSearch(resnetlstmgru,
                     objective='val_accuracy',
                     max_trials=MAX_TRAILS,
                     seed=seed,
                     overwrite=True
                     )

print("Search space summary:\n {}".format(tuner.search_space_summary()))
print("Searching starting >>> started")

tr_decay = 392
va_decay = 130
dir = '/content/drive/MyDrive/resnetlstmgru/logs/tuner/' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tuner.search(tr_data, epochs=epochs, steps_per_epoch=tr_decay//tr_data.batch_size,
             validation_data=va_data, verbose=1, validation_steps=va_decay//va_data.batch_size, 
             callbacks=[tf.keras.callbacks.TensorBoard(dir)])

print("Results summary:\n {}".format(tuner.results_summary()))
print("Top 10 best models:\n {}".format(tuner.get_best_models(10)))

best_hyperparameters = tuner.get_best_hyperparameters(num_trials = 1)[0]
best_model = tuner.hypermodel.build(best_hyperparameters)

best_model.save('/content/drive/MyDrive/resnetlstmgru/Model/')


In [ ]:
# model 4: cnnlstmgru
import datetime
MAX_TRAILS = 66
epochs = 10
seed = 40

# change the first parameter according to the network you want to train. 
tuner = RandomSearch(cnnlstmgru,
                     objective='val_accuracy',
                     max_trials=MAX_TRAILS,
                     seed=seed,
                     overwrite=True
                     )

print("Search space summary:\n {}".format(tuner.search_space_summary()))
print("Searching starting >>> started")

tr_decay = 392
va_decay = 130
dir = '/content/drive/MyDrive/cnnlstmgru/logs/tuner/' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tuner.search(tr_data, epochs=epochs, steps_per_epoch=tr_decay//tr_data.batch_size,
             validation_data=va_data, verbose=1, validation_steps=va_decay//va_data.batch_size, 
             callbacks=[tf.keras.callbacks.TensorBoard(dir)])

print("Results summary:\n {}".format(tuner.results_summary()))
print("Top 10 best models:\n {}".format(tuner.get_best_models(10)))

best_hyperparameters = tuner.get_best_hyperparameters(num_trials = 1)[0]
best_model = tuner.hypermodel.build(best_hyperparameters)

best_model.save('/content/drive/MyDrive/cnnlstmgru/Model/')


In [ ]:
# model 5: convlstm
# import datetime
# MAX_TRAILS = 1
# epochs = 10
# seed = 40

# # change the first parameter according to the network you want to train. 
# tuner = RandomSearch(convlstm,
#                      objective='val_accuracy',
#                      max_trials=MAX_TRAILS,
#                      seed=seed,
#                      overwrite=True
#                      )

# print("Search space summary:\n {}".format(tuner.search_space_summary()))
# print("Searching starting >>> started")

# tr_decay = 392
# va_decay = 130
# dir = '/content/drive/MyDrive/convlstm/logs/tuner/' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
# tuner.search(tr_data, epochs=epochs, steps_per_epoch=tr_decay//tr_data.batch_size,
#              validation_data=va_data, verbose=1, validation_steps=va_decay//va_data.batch_size, 
#              callbacks=[tf.keras.callbacks.TensorBoard(dir)])

# print("Results summary:\n {}".format(tuner.results_summary()))
# print("Top 10 best models:\n {}".format(tuner.get_best_models(10)))

# best_hyperparameters = tuner.get_best_hyperparameters(num_trials = 1)[0]
# best_model = tuner.hypermodel.build(best_hyperparameters)

# best_model.save('/content/drive/MyDrive/convlstm/Model/')


## Visualize using Tensorboard

In [ ]:
# %load_ext tensorboard
# !tensorboard dev upload --logdir /gdrive/MyDrive/logs/fit \
#   --name "When to Interupt" \
#   --description "Comparing the training results from different neural networks" \
#   --one_shot

# !tensorboard dev list

## Load the trained best model saved before.

## Train the best model with best hyperparameters from start

In [ ]:
from tensorflow.keras.models import load_model
#path = "/gdrive/MyDrive/Model/"
path = "/content/drive/MyDrive/cnnlstm/Model/"
cnnlstm_model = load_model(path)
cnnlstm_model.summary()
plot_model(model, to_file="cnnlstm_model.png", show_shapes=True, show_dtype=True, show_layer_names=True, rankdir="LR", expand_nested=True, dpi=96)


# tr_decay = 392
# tr_decay = 130
# dir = "/content/drive/MyDrive/cnnlstm/logs/fit/"
# cnnlstm_model.fit(tr_data, epochs=epochs, steps_per_epoch=tr_decay//tr_data.batch_size,
#              validation_data=va_data, verbose=1, validation_steps=tr_decay//va_data.batch_size, 
#              callbacks=[tf.keras.callbacks.TensorBoard(dir)])

In [ ]:
from tensorflow.keras.models import load_model
#path = "/gdrive/MyDrive/Model/"
path = "/content/drive/MyDrive/cnnlstmgru/Model/"
cnnlstmgru_model = load_model(path)
cnnlstmgru_model.summary()
plot_model(model, to_file="cnnlstmgru_model.png", show_shapes=True, show_dtype=True, show_layer_names=True, rankdir="LR", expand_nested=True, dpi=96)

tr_decay = 392
tr_decay = 130
dir = "/content/drive/MyDrive/cnnlstmgru/logs/fit/"
cnnlstmgru_model.fit(tr_data, epochs=epochs, steps_per_epoch=tr_decay//tr_data.batch_size,
             validation_data=va_data, verbose=1, validation_steps=tr_decay//va_data.batch_size, 
             callbacks=[tf.keras.callbacks.TensorBoard(dir)])

In [ ]:
from tensorflow.keras.models import load_model
#path = "/gdrive/MyDrive/Model/"
path = "/content/drive/MyDrive/resnetlstmgru/Model/"
resnetlstmgru_model = load_model(path)
resnetlstmgru_model.summary()
plot_model(model, to_file="resnetlstmgru_model.png", show_shapes=True, show_dtype=True, show_layer_names=True, rankdir="LR", expand_nested=True, dpi=96)


tr_decay = 392
tr_decay = 130
dir = "/content/drive/MyDrive/resnetlstmgru/logs/fit/"
resnetlstmgru_model.fit(tr_data, epochs=epochs, steps_per_epoch=tr_decay//tr_data.batch_size,
             validation_data=va_data, verbose=1, validation_steps=tr_decay//va_data.batch_size, 
             callbacks=[tf.keras.callbacks.TensorBoard(dir)])

In [ ]:
from tensorflow.keras.models import load_model
#path = "/gdrive/MyDrive/Model/"
path = "/content/drive/MyDrive/cnngru/Model/"
cnngru_model = load_model(path)
cnngru_model.summary()
plot_model(model, to_file="cnngru_model.png", show_shapes=True, show_dtype=True, show_layer_names=True, rankdir="LR", expand_nested=True, dpi=96)


tr_decay = 392
tr_decay = 130
dir = "/content/drive/MyDrive/cnngru/logs/fit/"
cnngru_model.fit(tr_data, epochs=epochs, steps_per_epoch=tr_decay//tr_data.batch_size,
             validation_data=va_data, verbose=1, validation_steps=tr_decay//va_data.batch_size, 
             callbacks=[tf.keras.callbacks.TensorBoard(dir)])

In [ ]:
# from tensorflow.keras.models import load_model
# #path = "/gdrive/MyDrive/Model/"
# path = "/content/drive/MyDrive/cnngru/Model/"
# cnngru_model = load_model(path)
# cnngru_model.summary()
# plot_model(model, to_file="cnngru_model.png", show_shapes=True, show_dtype=True, show_layer_names=True, rankdir="LR", expand_nested=True, dpi=96)


In [ ]:
# tr_decay = 392
# tr_decay = 130
# dir = "/content/drive/MyDrive/cnngru/logs/fit/"
# cnngru_model.fit(tr_data, epochs=epochs, steps_per_epoch=tr_decay//tr_data.batch_size,
#              validation_data=va_data, verbose=1, validation_steps=tr_decay//va_data.batch_size, 
#              callbacks=[tf.keras.callbacks.TensorBoard(dir)])

In [ ]:
# with 3 classes

# dir = "/gdrive/MyDrive/logs/fit/"
# final_model.fit(tr_data, epochs=epochs, steps_per_epoch=605//tr_data.batch_size,
#              validation_data=va_data, verbose=1, validation_steps=200//va_data.batch_size, 
#              callbacks=[tf.keras.callbacks.TensorBoard(dir)])

## Plot using Tensorboard

In [ ]:
!pip install -U tensorboard

In [ ]:
!tensorboard dev upload --logdir /content/drive/MyDrive/cnngru/logs/fit \
  --name "When to Interupt(cnngru)" \
  --description "Comparing the training results from different neural networks" \
  --one_shot

In [ ]:
!tensorboard dev list

In [ ]:
# for 3 classes 

# !tensorboard dev upload --logdir /gdrive/MyDrive/logs/fit \
#   --name "When to Interupt" \
#   --description "Comparing the training results from different neural networks" \
#   --one_shot

In [ ]:
# for 3 classes
# !tensorboard dev list